<a href="https://colab.research.google.com/github/FernandoElizarraras7/SIMULACION-II/blob/main/VARIABLES_DE_CONTROL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VARIABLES DE CONTROL

El método de Variables de Control es una técnica de reducción de varianza utilizada en simulaciones de Monte Carlo.
Se basa en aprovechar una variable auxiliar $Y$ que:

*   Está correlacionada con la variable de interés $X=f(U)$.
*   Tiene un valor esperado conocido $E[Y]$.


Si $X$ y $Y$ están positivamente correlacionadas, es posible disminuir las fluctuaciones del estimador de la integral al restar una fracción del término
$Y-E[Y]$, que tiene media cero pero se comporta similar a $X$.

El nuevo estimador se define como:
$$I_{VC} = \overline{X}-b^*(\overline{Y}-E[Y])$$
donde:
$$b^* = \frac{Cov(X, Y)}{Var(Y)}$$
Este valor $b^*$ es el coeficiente óptimo que minimiza la varianza del estimador.

La ventaja principal es que, si $Y$ y $X$ están bien correlacionadas, el término de corrección $b^*(Y-E[Y])$ cancela parte del ruido estadístico de $X$, reduciendo significativamente la varianza sin aumentar el número de simulaciones.

In [15]:
# MC Crudo vs Variables de Control

import numpy as np
import time

# Se define la función
def f(x):
  return 1 / (1 + x**2)

# Número de muestras
N = 10**5

# === Monte Carlo Crudo ===
t1_start = time.time()

U = np.random.rand(N)  # Muestras en [0,1]
X = f(U)  # Se evalúa en f(x)
I_crudo = np.mean(X)  # Promedio
Var_crudo = np.var(X, ddof=1) / N  # Varianza del estimador
Std_crudo = np.sqrt(Var_crudo)  # Desviación estándar

t1_end = time.time()
t1 = t1_end - t1_start

print("=== Método Crudo ===")
print(f"Estimación Integral = {I_crudo:.6f}")
print(f"Varianza = {Var_crudo:.6e}")
print()

# === Variables de Control ===
t2_start = time.time()

Y = U  # La variable de control es U porque Y = promedio de U_i y cada U_i ~ U(0,1)
EY = 0.5  # Valor esperado de U(0,1)

# Se calcula coeficiente óptimo b*
cov_XY = np.cov(X, Y, ddof=1)[0, 1]
var_Y = np.var(Y, ddof=1)
b_opt = cov_XY / var_Y

# Estimador con variable de control
I_control = np.mean(X - b_opt * (Y - EY))
Var_control = np.var(X - b_opt * (Y - EY), ddof=1) / N
Std_control = np.sqrt(Var_control)

t2_end = time.time()
t2 = t2_end - t2_start

print("=== Método Variables de Control ===")
print(f"Estimación Integral = {I_control:.6f}")
print(f"Coeficiente b* = {b_opt:.6f}")
print(f"Varianza = {Var_control:.6e}")
print()

# === Reducción de la Varianza ===
reducción = (1 - Var_control / Var_crudo) * 100
print("=== Reducción de la Varianza ===")
print(f"Reducción de la Varianza = {reducción:.2f}%")


=== Método Crudo ===
Estimación Integral = 0.785104
Varianza = 2.575793e-07

=== Método Variables de Control ===
Estimación Integral = 0.785492
Coeficiente b* = -0.553842
Varianza = 3.162219e-09

=== Reducción de la Varianza ===
Reducción de la Varianza = 98.77%


## CONCLUSIONES

*   La técnica de Variables de Control aprovecha la correlación entre la variable de interés $f(U)$ y una variable auxiliar $Y$ con valor esperado conocido, para reducir la varianza del estimador sin incrementar el número de simulaciones.
*   En términos prácticos, el estimador con variables de control logra mayor precisión con la misma cantidad de muestras, o la misma precisión con muchas menos simulaciones, incrementando notablemente la eficiencia del método Monte Carlo.
*   En este caso, al elegir $Y=U$ y $E[Y]=0.5$, se obtuvo una reducción relativa de varianza del 98.77 %, lo que demuestra una altísima eficiencia del método.
*   La reducción se debe a la relación inversa entre $f(U)$ y $U$; cuando $U$ aumenta, $f(U)$ disminuye, lo que genera una fuerte correlación aprovechada por el término correctivo del estimador.
*   La técnica es especialmente útil cuando se dispone de una buena variable auxiliar con media conocida y correlación significativa con la función integrando.





